# Polygon Stock Market API Tools

>[Polygon](https://polygon.io/) The Polygon.io Stocks API provides REST endpoints that let you query the latest market data from all US stock exchanges.

This notebook uses tools to get stock market data like the latest quote and news for a ticker from Polygon.

In [ ]:
import getpass
import os

os.environ["POLYGON_API_KEY"] = getpass.getpass()

In [24]:
from langchain_community.tools.polygon.last_quote import PolygonLastQuote
from langchain_community.tools.polygon.ticker_news import PolygonTickerNews
from langchain_community.utilities.polygon import PolygonAPIWrapper

In [25]:
api_wrapper = PolygonAPIWrapper()
ticker = "AAPL"

### Get latest quote for ticker

In [55]:
# Get the last quote for ticker
last_quote_tool = PolygonLastQuote(api_wrapper=api_wrapper)
last_quote = last_quote_tool.run(ticker)
print(f"Tool output: {last_quote}")

Tool output: {"P": 180.99, "S": 5, "T": "AAPL", "X": 11, "i": [604], "p": 180.98, "q": 31662815, "s": 2, "t": 1708445246516556649, "x": 21, "y": 1708445246516369924, "z": 3}


In [29]:
import json

# Convert the last quote response to JSON
last_quote = last_quote_tool.run(ticker)
last_quote_json = json.loads(last_quote)

In [33]:
# Print the latest price for ticker
latest_price = last_quote_json["p"]
print(f"Latest price for {ticker} is ${latest_price}")

Latest price for AAPL is $180.74


### Get latest news for ticker

In [58]:
ticker_news_tool = PolygonTickerNews(api_wrapper=api_wrapper)
ticker_news = ticker_news_tool.run(ticker)
print(f"Tool output: {ticker_news}")

Tool output: [{"id": "RckckqTS-K2zI6WRyycBB0HonfGLc2MPTQROWsbpKSA", "publisher": {"name": "The Motley Fool", "homepage_url": "https://www.fool.com/", "logo_url": "https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg", "favicon_url": "https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico"}, "title": "Should You Worry About Apple's Slowdown in China?", "author": "newsfeedback@fool.com (Adria Cimino)", "published_utc": "2024-02-20T15:10:00Z", "article_url": "https://www.fool.com/investing/2024/02/20/should-you-worry-about-apples-slowdown-in-china/", "tickers": ["AAPL"], "image_url": "https://g.foolcdn.com/editorial/images/765910/aapl.png", "description": "A local smartphone giant is gaining market share.", "keywords": ["investing"]}, {"id": "5goYCKw3ZsHlJYwN1XX8AJCyybZuvfJeoUz38SzlYBM", "publisher": {"name": "The Motley Fool", "homepage_url": "https://www.fool.com/", "logo_url": "https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg", "favicon_url": "h

In [57]:
# Convert the news response to JSON array
ticker_news_json = json.loads(ticker_news)
print(f"Total news items: {len(ticker_news_json)}")

Total news items: 10


In [56]:
# Inspect the first news item
news_item = ticker_news_json[0]
print(f"Title: {news_item['title']}")
print(f"Description: {news_item['description']}")
print(f"Publisher: {news_item['publisher']['name']}")
print(f"URL: {news_item['article_url']}")

Title: Should You Worry About Apple's Slowdown in China?
Description: A local smartphone giant is gaining market share.
Publisher: The Motley Fool
URL: https://www.fool.com/investing/2024/02/20/should-you-worry-about-apples-slowdown-in-china/
